# Aggregate Data Creation

In [6]:
import pandas as pd

In [ ]:
# %%capture
#@title Setup Environment
# Install the latest Tensorflow version.
!pip install "tensorflow-text==2.8.*" -q
!pip install bokeh -q  
!pip install simpleneighbors[annoy] -q
!pip install tqdm -q

     |████████████████████████████████| 4.9 MB 32.5 MB/s 
     |████████████████████████████████| 462 kB 70.1 MB/s 
     |████████████████████████████████| 646 kB 39.6 MB/s 


In [ ]:

from multiprocessing import Pool
from simpleneighbors import SimpleNeighbors
from tensorflow_text import SentencepieceTokenizer
from tqdm import tqdm, trange
from tqdm.notebook import tqdm
import bokeh
import bokeh.models
import bokeh.plotting
import codecs
import numpy as np
import os
import pandas as pd
import random
import sklearn.metrics.pairwise
import string
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
tqdm.pandas()

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
os.chdir("./drive/MyDrive/NLP Project")

In [3]:
path="./splitData"
# os.chdir("./drive/MyDrive/NLP Project")


In [ ]:

def parallelize_dataframe(df, func, n_cores=2):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [7]:
df_censored=pd.read_csv(path+"/censored.csv",names=["mid","text","created_at","deleted_last_seen","permission_denied"
]).dropna()
df_censored = df_censored.drop(df_censored.index[df_censored['text'] == "text"])

In [8]:
len_censored=len(df_censored)

In [9]:
len_censored

134417

In [ ]:
cols=["mid","text","created_at","deleted_last_seen","permission_denied"]

In [ ]:
df_uncensored=pd.read_csv(path+"/not_censored.csv",names=["mid","text","created_at","deleted_last_seen","permission_denied"
],nrows=len_censored*4).dropna()

## emdeddings GUSE

In [ ]:
# The 16-language multilingual module is the default but feel free
# to pick others from the list and compare the results.
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3' #@param ['https://tfhub.dev/google/universal-sentence-encoder-multilingual/3', 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3']

model = hub.load(module_url)

def embed_text(input):
    return model(input)[-1]

def embed_text_list(input):
    return model(input).numpy()

In [ ]:
def parallel_guse(df):
    #df["guse"] = df["text"].progress_apply(la?mbda x:embed_text(str(x)))
    df["guse"] = embed_text_list(df["text"].to_list()).tolist()
    return df

In [ ]:
def serial_dataframe(df, func, n_splits=2):
    df_split = np.array_split(df, n_splits)
    df = pd.DataFrame()
    for df_s in tqdm(df_split):
        df_rs=func(df_s)
        df = pd.concat([df,df_s])
    return df

In [ ]:
# df_censored=serial_dataframe(df=df_censored, func=parallel_guse, n_splits=int(len_censored/200))

  0%|          | 0/672 [00:00<?, ?it/s]

In [ ]:
# df_censored.to_pickle(path+"/censored.pickle")

In [ ]:
# df_uncensored=serial_dataframe(df=df_uncensored, func=parallel_guse, n_splits=int(len_censored/200)*400)

  0%|          | 0/268800 [00:00<?, ?it/s]

In [ ]:
# df_uncensored.to_pickle(path+"/not_censored.pickle")

In [ ]:
# df_data=pd.concat([df_censored.loc[:,['permission_denied','guse']],df_uncensored.loc[:,['permission_denied','guse']]])

In [ ]:
# df_data["permission_denied"]=df_data["permission_denied"].astype(bool).astype(int)

In [ ]:
# df_data.to_pickle(path+"/Large_embeddings_data.pickle")

In [ ]:
# X=df_data["guse"].to_numpy()
# X=np.vstack(X)
# y=df_data["permission_denied"].values

In [ ]:
import pandas as pd

In [ ]:
df_data=pd.read_pickle(path+"/Large_embeddings_data.pickle")

In [ ]:
# with open(path+'/X_data.npy', 'wb') as f:
#     np.save(f,X)
# with open(path+'/Y_data.npy', 'wb') as f:
#     np.save(f,y)

In [ ]:
# del df_data

In [ ]:
# import sys
# def sizeof_fmt(num, suffix='B'):
#     ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
#     for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
#         if abs(num) < 1024.0:
#             return "%3.1f %s%s" % (num, unit, suffix)
#         num /= 1024.0
#     return "%.1f %s%s" % (num, 'Yi', suffix)

# for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
#                          key= lambda x: -x[1])[:10]:
#     print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                       df_data:  3.0 GiB
                          _i17:  587.0 B
                           _ih:  272.0 B
                            In:  272.0 B
                             _:  272.0 B
                            _6:  272.0 B
                           _oh:  248.0 B
                           Out:  248.0 B
                            __:  200.0 B
                            _5:  200.0 B


# Sampling Methods

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("./drive/MyDrive/NLP Project")
path="./splitData"


Mounted at /content/drive


In [ ]:
 os.listdir(path)

['estim.pickle',
 'embeding_data.pickle',
 'autokeras.pickle',
 'old',
 'censored.csv',
 'not_censored.csv',
 'censored.pickle',
 'not_censored.pickle',
 'Large_embeddings_data.pickle',
 'Y_data.npy',
 'X_data.npy',
 'Copy of estim.pickle',
 'Copy of autokeras.pickle',
 'Copy of Large_embeddings_data.pickle',
 'Copy of censored.csv',
 'Copy of censored.pickle',
 'Copy of not_censored.pickle',
 'Copy of not_censored.csv',
 'Copy of embeding_data.pickle',
 'Copy of Y_data.npy',
 'Copy of X_data.npy']

## sampling using SMOTE

In [ ]:
import numpy as np

In [ ]:
# with open(path+'/X_data.npy', 'rb') as f:
#     X=np.load(f)
# with open(path+'/Y_data.npy', 'rb') as f:
#     y=np.load(f)

In [ ]:
# del X
# del y
# del df_data

## sampling using Translation

In [ ]:
# https://py-googletrans.readthedocs.io/en/latest/

In [ ]:
import pandas as pd

In [ ]:
!pip install googletrans==4.0.0-rc1
!pip install sentencepiece
!pip install transformers datasets -q

  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17416 sha256=e42e10b19d5208af7c02fb99ca37d1de78e554d40eae503826d1938c059ff02a
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


### language model



In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

In [ ]:
def translate(text,source,destination):
  tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_1.2B", src_lang=source)

  model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_1.2B")
  encoded_zh = tokenizer(text, return_tensors="pt")
  generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id(destination))
  data=tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
  return data[-1]

In [ ]:
%%time
translate("how are you. I am doing well","en","zh")

CPU times: user 17.5 s, sys: 2.75 s, total: 20.2 s
Wall time: 22.9 s


'你怎么样?我做得很好。'

In [ ]:
def sample_text(text):
  zh_en=translate(text,"en","zh")
  en_zh=translate(zh_en,"zh","en")
  return en_zh

In [ ]:
df_censored=pd.read_pickle(path+"/censored.pickle")

In [ ]:
# df_uncensored=pd.read_pickle(path+"/not_censored.pickle",nrows)

In [ ]:
df_censored.head()

,mid,text,created_at,deleted_last_seen,permission_denied,guse
0,mkTL1pvXTN,这王某人与中国青年报怎么这么有缘啊。uRLOGZLR4： 原来是个大流氓,2012-01-07 22:29:56,2012-03-03 03:35:43.903155,True,"[0.006624801550060511, 0.031350668519735336, 0..."
1,m5PIm1JzG8,经常看到有代表四处调研，你代表的是哪个行业，就好好调研一下本行业的问题好不好？自己的工作都做...,2012-01-05 09:00:03,2012-01-30 21:36:44.229791,True,"[0.03378220275044441, -0.007479660212993622, 0..."
2,mJGNyWD5fj,刚在宝宝的书架上发现这本书，翻了一下，历史事实描述一派胡言。问宝宝，这本书的内容你们考试吗？...,2012-01-02 12:45:58,False,True,"[0.009367837570607662, -0.03505528345704079, -..."
3,m3e5KqAhFh,应该找uRLOWL0ZX： 去影他相！新年流流揸公车出来行，仲要违章，真是岂有此理！,2012-01-03 18:15:24,2012-01-30 19:15:54.549843,True,"[-0.02759631723165512, -0.009455885738134384, ..."
4,mkTLyULvfH,崔永元：微博春晚节目——天津快板：竹板这么一打啊，啥也不能说，不能说食品，不能说动车，不能说...,2012-01-07 12:02:02,2012-02-28 11:08:24.583212,True,"[0.002411450492218137, 0.025735968723893166, 0..."


In [ ]:
text_data=df_censored.text[1]

In [ ]:
text_data

'经常看到有代表四处调研，你代表的是哪个行业，就好好调研一下本行业的问题好不好？自己的工作都做不了还代表什么代表谁呢？uB5N2XGJG：  哈哈'

In [ ]:
# from googletrans import Translator
# translator = Translator()

# def text_sample(text_data):
#   ch_en=translator.translate(text_data,dest='en')
#   src_in=ch_en.src
#   en_ch=translator.translate(ch_en.text,dest=src_in)
#   return en_ch.text

In [ ]:
df_censored_sample_1=df_censored.copy()

In [ ]:
df_censored_sample_1.head()

,mid,text,created_at,deleted_last_seen,permission_denied,guse
0,mkTL1pvXTN,这王某人与中国青年报怎么这么有缘啊。uRLOGZLR4： 原来是个大流氓,2012-01-07 22:29:56,2012-03-03 03:35:43.903155,True,"[0.006624801550060511, 0.031350668519735336, 0..."
1,m5PIm1JzG8,经常看到有代表四处调研，你代表的是哪个行业，就好好调研一下本行业的问题好不好？自己的工作都做...,2012-01-05 09:00:03,2012-01-30 21:36:44.229791,True,"[0.03378220275044441, -0.007479660212993622, 0..."
2,mJGNyWD5fj,刚在宝宝的书架上发现这本书，翻了一下，历史事实描述一派胡言。问宝宝，这本书的内容你们考试吗？...,2012-01-02 12:45:58,False,True,"[0.009367837570607662, -0.03505528345704079, -..."
3,m3e5KqAhFh,应该找uRLOWL0ZX： 去影他相！新年流流揸公车出来行，仲要违章，真是岂有此理！,2012-01-03 18:15:24,2012-01-30 19:15:54.549843,True,"[-0.02759631723165512, -0.009455885738134384, ..."
4,mkTLyULvfH,崔永元：微博春晚节目——天津快板：竹板这么一打啊，啥也不能说，不能说食品，不能说动车，不能说...,2012-01-07 12:02:02,2012-02-28 11:08:24.583212,True,"[0.002411450492218137, 0.025735968723893166, 0..."


In [ ]:
# df_censored=serial_dataframe(df=df_censored, func=parallel_guse, n_splits=int(len_censored/200))

In [ ]:
def apply_translation(df):
  df["text"]=df["text"].progress_apply(lambda x:sample_text(str(x)))
  return df

In [ ]:
# def upsample_df(df):
#   df=apply_translation(df)
#   df=serial_dataframe(df=df, func=parallel_guse, n_splits=int(len_censored/200))
#   return df

In [ ]:
# https://stackoverflow.com/questions/62728985/how-do-i-translate-using-huggingface-from-chinese-to-english

In [ ]:
# df_censored_sample_1=parallelize_dataframe(df_censored_sample_1[:4],apply_translation)

In [ ]:
# df_censored_sample_1=apply_translation(df_censored_sample_1)

In [ ]:
# df_censored_sample_1=upsample_df(df_censored_sample_1)

In [ ]:
# https://huggingface.co/docs/transformers/multilingual#xlm-language-embeddings

In [ ]:
# !pip install sentencepiece

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

Downloading:   0%|          | 0.00/3.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/271 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/909 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.62G [00:00<?, ?B/s]

['Often see there are representatives around the research, which industry you represent, do a good study of the problem of the industry, can not do your own work and who represents what? uB5N2XGJG: ha']

In [ ]:
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en")

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

In [ ]:
encoded_zh = tokenizer("['Often see there is a representative of four fields of research, which industry you represent, so well to investigate the issue of this industry? your work cannot be done and what does it represent? uB5N2XGJG: haha']", return_tensors="pt")

In [ ]:
generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("zh"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

["[‘Often看,有四个研究领域的代表,你代表哪个行业,那么好来调查这个行业的问题?你的工作不能完成,它代表什么? uB5N2XGJG: haha']"]

# Topic Modelling